In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)|

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/mit-bh-arrythmia-org'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/848.hea
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/850.hea
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/888.dat
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/808.hea
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/842.hea-
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/891.dat
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/894.atr
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/885.dat
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/871.atr
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/883.xws
/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular

In [2]:
files = []

for dirname,_,filenames in os.walk('/kaggle/input/mit-bh-arrythmia-org'):
    for filename in filenames:
        if filename.lower().endswith(('.atr', '.hea','.dat','.xms')):
            path = os.path.join(dirname, filename)
            files.append(path)

In [3]:
files

['/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/848.hea',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/850.hea',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/888.dat',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/808.hea',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/891.dat',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/894.atr',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/885.dat',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/871.atr',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/827.hea',
 '/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0/847.hea',
 '/kaggle/input/mit-

## Loading data

In [4]:
!pip install wfdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 6.9 MB/s eta 0:00:00


In [5]:
import os
import wfdb

def load_record(record_base, folder):
    """
    Loads a single ECG record and its annotation.
    
    Parameters:
        record_base (str): Base name of the record (e.g., '100')
        folder (str): Path to the folder containing the record files (.hea, .dat, .atr)
    
    Returns:
        tuple: (record, annotation) where record is from .hea and .dat, and annotation is from .atr.
    """
    record_path = os.path.join(folder, record_base)
    # Load the ECG record (automatically loads .hea and .dat)
    record = wfdb.rdrecord(record_path)
    # Load the annotation (assumed to be of type 'atr')
    annotation = wfdb.rdann(record_path, 'atr')
    return record, annotation

def load_all_records(folder, record_list=None):
    """
    Loads all records (.hea, .dat, and .atr) from the specified folder.
    
    Parameters:
        folder (str): Path to the dataset folder.
        record_list (list, optional): List of record base names to load. If None,
                                      it will search the folder for all .hea files.
    
    Returns:
        dict: A dictionary mapping record base names to a dict with keys 'record' and 'annotation'.
    """
    records = {}
    # If no record_list is provided, find all .hea files in the folder.
    if record_list is None:
        record_list = [os.path.splitext(f)[0] for f in os.listdir(folder) if f.lower().endswith('.hea')]
    
    for rec in record_list:
        try:
            rec_obj, annot_obj = load_record(rec, folder)
            records[rec] = {'record': rec_obj, 'annotation': annot_obj}
            print(f"Loaded record: {rec}")
        except Exception as e:
            print(f"Error loading record {rec}: {e}")
    
    return records

if __name__ == '__main__':
    folder_path = "/kaggle/input/mit-bh-arrythmia-org/mit-bih-supraventricular-arrhythmia-database-1.0.0"
    
    # Optionally, provide a list of record base names; otherwise, it auto-discovers them.
    # record_names = ['100', '101', '102']  # for example
    loaded_records = load_all_records(folder_path)
    
    # Now, loaded_records is a dictionary in memory where each key is a record name,
    # and its value is a dictionary containing the loaded record and annotation.
    print(f"Total records loaded: {len(loaded_records)}")


Loaded record: 848
Loaded record: 850
Loaded record: 808
Loaded record: 827
Loaded record: 847
Loaded record: 844
Loaded record: 882
Loaded record: 883
Loaded record: 885
Loaded record: 852
Loaded record: 891
Loaded record: 884
Loaded record: 874
Loaded record: 860
Loaded record: 841
Loaded record: 875
Loaded record: 822
Loaded record: 820
Loaded record: 807
Loaded record: 881
Loaded record: 846
Loaded record: 890
Loaded record: 845
Loaded record: 812
Loaded record: 857
Loaded record: 842
Loaded record: 880
Loaded record: 864
Loaded record: 828
Loaded record: 871
Loaded record: 856
Loaded record: 892
Loaded record: 851
Loaded record: 800
Loaded record: 858
Loaded record: 821
Loaded record: 862
Loaded record: 870
Loaded record: 877
Loaded record: 840
Loaded record: 868
Loaded record: 855
Loaded record: 810
Loaded record: 859
Loaded record: 888
Loaded record: 824
Loaded record: 825
Loaded record: 805
Loaded record: 876
Loaded record: 873
Loaded record: 823
Loaded record: 867
Loaded recor

## Preprocessing

In [6]:
import os
import wfdb
import numpy as np
from scipy import signal

# Step 1. Downsample the signal to a target sampling frequency (256 Hz)
def downsample_signal(ecg_signal, orig_fs, target_fs=256):
    """
    Downsamples the ECG signal from orig_fs to target_fs.
    ecg_signal: numpy array, shape (n_samples, n_channels) or (n_samples,)
    Returns the downsampled signal.
    """
    if orig_fs == target_fs:
        return ecg_signal
    factor = orig_fs / target_fs
    num_samples = int(np.ceil(len(ecg_signal) / factor))
    if ecg_signal.ndim == 1:
        downsampled = signal.resample(ecg_signal, num_samples)
    else:
        downsampled = np.zeros((num_samples, ecg_signal.shape[1]))
        for ch in range(ecg_signal.shape[1]):
            downsampled[:, ch] = signal.resample(ecg_signal[:, ch], num_samples)
    return downsampled

# Step 2. Bandpass filter the signal (0.5 Hz to 40 Hz)
def bandpass_filter(ecg_signal, fs, lowcut=0.5, highcut=40.0, order=4):
    """
    Applies a Butterworth bandpass filter to the ECG signal.
    """
    nyquist = 0.5 * fs
    if highcut >= nyquist:
        new_high = nyquist * 0.99
        print(f"Warning: highcut ({highcut} Hz) is >= Nyquist frequency ({nyquist} Hz). Adjusting highcut to {new_high:.2f} Hz.")
        highcut = new_high
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    filtered_signal = signal.filtfilt(b, a, ecg_signal, axis=0)
    return filtered_signal

# Step 3. Segment the signal into 3-beat windows using R-peak annotations
def segment_signal(ecg_signal, r_peaks, window_beats=3, margin=0, pad_to_max=True):
    """
    Segments the ECG signal into windows covering 'window_beats' beats using R-peak annotations.
    
    Parameters:
      ecg_signal: numpy array, the preprocessed ECG signal.
      r_peaks: numpy array of R-peak indices (already adjusted for downsampling).
      window_beats: number of consecutive beats to include in each segment (default is 3).
      margin: extra samples to include before the first beat and after the last beat.
      pad_to_max: if True, pad each segment with zeros to the maximum segment length across all segments.
      
    Returns:
      segments: a list of numpy arrays, each of which is a segment corresponding to a 3-beat window,
                padded to a fixed length if pad_to_max is True.
    """
    segments = []
    # Extract raw segments based on consecutive R-peaks.
    for i in range(len(r_peaks) - window_beats + 1):
        start = max(r_peaks[i] - margin, 0)
        end = r_peaks[i + window_beats - 1] + margin
        segment = ecg_signal[start:end]
        segments.append(segment)
    
    # If required, pad all segments to have the same fixed length (maximum segment length)
    if pad_to_max and segments:
        max_len = max(segment.shape[0] for segment in segments)
        padded_segments = []
        for seg in segments:
            # Pad at the end with zeros if seg is shorter than max_len
            if seg.shape[0] < max_len:
                pad_width = max_len - seg.shape[0]
                # For 1D signal, pad with zeros on the right
                seg = np.pad(seg, (0, pad_width), mode='constant', constant_values=0)
            padded_segments.append(seg)
        segments = padded_segments
        
    return segments
    

# Step 4. Apply Short-Time Fourier Transform (STFT) on each segment
def apply_stft(segment, fs, nperseg=64, noverlap=32):
    """
    Applies Short-Time Fourier Transform (STFT) to the given ECG segment.
    
    If the segment length is shorter than nperseg, nperseg (and noverlap) is automatically
    adjusted to the segment length to avoid warnings.
    
    Parameters:
      segment: 1D numpy array containing the ECG segment.
      fs: Sampling frequency.
      nperseg: Length of each segment for STFT.
      noverlap: Number of points to overlap between segments.
      
    Returns:
      spectrogram: Magnitude spectrogram as a numpy array.
    """
    # Adjust nperseg if segment is shorter than the default nperseg.
    seg_len = len(segment)
    if seg_len < nperseg:
        nperseg = seg_len
        noverlap = int(seg_len * 0.5)  # For example, use 50% overlap
    
    # Compute STFT
    f, t, Zxx = signal.stft(segment, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)
    return spectrogram


# New master function to preprocess loaded records
def preprocess_loaded_records(loaded_records, target_fs=256):
    """
    Processes already loaded ECG records (a dictionary with keys as record names and values as
    dictionaries containing 'record' and 'annotation').
    
    For each record:
      1. Downsamples the raw signal (from record.p_signal) to target_fs.
      2. Applies a Butterworth bandpass filter (0.5 - 40 Hz).
      3. Adjusts R-peak indices from the annotation (annot.sample) and segments the signal
         into 3-beat windows.
      4. Applies STFT on each segment.
      
    Returns a dictionary mapping record names to lists of spectrograms.
    """
    results = {}
    for rec_name, data in loaded_records.items():
        record = data['record']
        annot = data['annotation']
        
        # Get the raw signal and original sampling frequency.
        ecg_signal = record.p_signal  # shape: (n_samples, n_channels)
        orig_fs = record.fs

        # Process only the first channel for simplicity.
        ecg_signal = ecg_signal[:, 0]

        # Step 1: Downsample the signal.
        downsampled = downsample_signal(ecg_signal, orig_fs, target_fs=target_fs)

        # Step 2: Apply bandpass filtering.
        filtered = bandpass_filter(downsampled, fs=target_fs)

        # Adjust R-peak indices from the annotation (they are for the original signal)
        factor = orig_fs / target_fs
        r_peaks = (annot.sample / factor).astype(int)

        # Step 3: Segment the signal into 3-beat windows.
        segments = segment_signal(filtered, r_peaks, window_beats=3, margin=0)

        # Step 4: Apply STFT on each segment.
        spectrograms = []
        for seg in segments:
            spec = apply_stft(seg, fs=target_fs)
            spectrograms.append(spec)

        results[rec_name] = spectrograms
        print(f"Processed record {rec_name}: {len(segments)} segments")
    
    return results

In [7]:
 processed_data = preprocess_loaded_records(loaded_records, target_fs=64)

Processed record 848: 4287 segments
Processed record 850: 1838 segments
Processed record 808: 1775 segments
Processed record 827: 1860 segments
Processed record 847: 1834 segments
Processed record 844: 1707 segments
Processed record 882: 1933 segments
Processed record 883: 1820 segments
Processed record 885: 2052 segments
Processed record 852: 2663 segments
Processed record 891: 2777 segments
Processed record 884: 3020 segments
Processed record 874: 2238 segments
Processed record 860: 2461 segments
Processed record 841: 1852 segments
Processed record 875: 2050 segments
Processed record 822: 2335 segments
Processed record 820: 2354 segments
Processed record 807: 1952 segments
Processed record 881: 2300 segments
Processed record 846: 1683 segments
Processed record 890: 2172 segments
Processed record 845: 2885 segments
Processed record 812: 1871 segments
Processed record 857: 2608 segments
Processed record 842: 2557 segments
Processed record 880: 3485 segments
Processed record 864: 1905 s

## Model

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# Custom collate function for dynamic padding along the time dimension.
def pad_collate(batch):
    """
    Pads each spectrogram in the batch so that they all have the same time dimension.
    Each sample is a tuple (spectrogram, label) where:
      - spectrogram is a tensor of shape [1, freq_bins, time_frames]
      - label is a tensor (dummy label in our case)
    """
    if len(batch) == 0:
        return torch.empty(0), torch.empty(0, dtype=torch.long)
    
    max_time = max(sample[0].shape[2] for sample in batch)
    padded_batch = []
    for spec, label in batch:
        time_diff = max_time - spec.shape[2]
        # Pad along the time dimension (last dimension)
        padded_spec = F.pad(spec, (0, time_diff), mode='constant', value=0)
        padded_batch.append((padded_spec, label))
    specs = torch.stack([item[0] for item in padded_batch], dim=0)
    # Ensure labels remain 1D
    labels = torch.stack([item[1] for item in padded_batch], dim=0).view(-1)
    return specs, labels

# ====================================================
# Custom Dataset for Preprocessed ECG Spectrograms
# ====================================================
class ECGSpectrogramDataset(Dataset):
    def __init__(self, processed_data, transform=None):
        """
        Args:
          processed_data (dict): Dictionary mapping record names to a list of spectrograms (numpy arrays).
                                  Each spectrogram is assumed to be of shape (freq_bins, time_frames).
          transform (callable, optional): A function/transform to apply to each sample.
        """
        self.samples = []
        for rec_name, spec_list in processed_data.items():
            for spec in spec_list:
                # Each spectrogram is a 2D numpy array; add channel dimension -> (1, freq_bins, time_frames)
                spec = np.expand_dims(spec, axis=0)
                # Dummy label (0) as placeholder; replace with actual label extraction.
                self.samples.append((spec.astype(np.float32), 0))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        spec, label = self.samples[idx]
        if self.transform:
            spec = self.transform(spec)
        return torch.tensor(spec), torch.tensor(label, dtype=torch.long)

# ====================================================
# Model Architecture: Baseline Model (as per paper, in PyTorch)
# ====================================================
class ECGBaselineModel(nn.Module):
    def __init__(self, in_freq, num_classes=5, hidden_size=1760):
        """
        Baseline model architecture:
          - Two convolution layers with filter sizes (3x11) and (3x3) respectively.
          - Each conv layer is followed by BatchNorm and ReLU activation.
          - The output is reshaped so that the time dimension becomes the sequence length.
          - Two stacked LSTM layers process the sequence.
          - The last hidden state is passed through a fully connected layer to produce logits.
        
        Args:
            in_freq (int): Frequency dimension of the input spectrogram (after STFT and padding).
                           For example, if your spectrograms are of shape (freq_bins, time_frames)
                           and freq_bins is 33, set in_freq=33.
            num_classes (int): Number of output classes.
            hidden_size (int): Hidden state size for the LSTM layers.
        """
        super(ECGBaselineModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=11, kernel_size=(3, 11), stride=1, padding=(1, 5))
        self.bn1 = nn.BatchNorm2d(11)
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=11, out_channels=11, kernel_size=(3, 3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(11)
        self.act2 = nn.ReLU()
        
        # Expected conv output shape: [batch, 11, in_freq, T_out]
        feature_dim = 11 * in_freq
        
        # LSTM layers: layout 'NTC' expects (batch, time, features)
        self.lstm = nn.LSTM(input_size=feature_dim, hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        """
        Forward pass.
        Args:
            x: input tensor of shape [batch, 1, freq_bins, time_frames]
        Returns:
            logits: tensor of shape [batch, num_classes]
        """
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)
        # x shape: [batch, 11, F_out, T_out]
        batch_size, channels, freq, time = x.size()
        # Reshape: collapse channels and freq into one dimension, treat time as sequence length.
        x = x.view(batch_size, channels * freq, time).transpose(1, 2)
        lstm_out, (hn, cn) = self.lstm(x)
        last_hidden = hn[-1]  # [batch, hidden_size]
        logits = self.fc(last_hidden)
        return logits

# ====================================================
# Training and Validation Functions
# ====================================================
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)
        total += labels.size(0)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for data, labels in val_loader:
            # Skip empty batches if any.
            if data.size(0) == 0:
                continue
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * data.size(0)
            total += labels.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
    epoch_loss = running_loss / total if total > 0 else 0
    epoch_acc = correct / total if total > 0 else 0
    return epoch_loss, epoch_acc

# ====================================================
# Main Training Loop: Display Train & Val Accuracy and Save Best Model
# ====================================================
if __name__ == '__main__':
    # Assume processed_data is already generated by your preprocessing pipeline.
    # processed_data should be a dict mapping record names to lists of spectrograms.
    
    # Create the full dataset.
    full_dataset = ECGSpectrogramDataset(processed_data)
    print(f"Total samples: {len(full_dataset)}")
    
    # Split dataset into training (80%) and validation (20%).
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=pad_collate)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=pad_collate)
    
    # Set device: use GPU if available, otherwise CPU.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # IMPORTANT: Set in_freq to the actual frequency dimension of your spectrograms.
    # For example, if your STFT outputs spectrograms with shape (freq_bins, time_frames) and freq_bins is 33,
    # then set in_freq = 33.
    in_freq = 33  # Adjust as needed.
    
    model = ECGBaselineModel(in_freq=in_freq, num_classes=5, hidden_size=1760)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    num_epochs = 20
    best_val_acc = 0.0
    save_path = "best_ecg_model.pth"
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc*100:.2f}%")
        
        # Save the best model based on validation accuracy.
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(f"  -> New best model saved with Val Acc: {val_acc*100:.2f}%")
    
    print(f"Training complete. Best validation accuracy: {best_val_acc*100:.2f}%")


Total samples: 187721
Train samples: 150176, Val samples: 37545
Epoch 1/20:
  Train Loss: 0.0004, Train Acc: 99.99%
  Val   Loss: 0.0000, Val   Acc: 100.00%
  -> New best model saved with Val Acc: 100.00%
Epoch 2/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
Epoch 3/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
Epoch 4/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
Epoch 5/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
Epoch 6/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
Epoch 7/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
Epoch 8/20:
  Train Loss: 0.0000, Train Acc: 100.00%
  Val   Loss: 0.0000, Val   Acc: 100.00%
